In [6]:
import requests
from bs4 import BeautifulSoup

iphone_url = "https://www.ebay.co.uk/sch/i.html?_from=R40&_nkw=iphone&_sacat=0&_pgn="
urls = list()

for i in range(500):
    print(i)
    try:
        search_url = iphone_url + str(i+1)

        response = requests.get(search_url)
        soup = BeautifulSoup(response.content, "html.parser")

        temp = [a["href"] for a in soup.find_all("a", class_="s-item__link")]

        urls = urls + temp
    except:
        break

    # Filter out non-product links
urls = [link for link in urls if "ebay.co.uk/itm/" in link]
urls = list(set(urls))
urls


['https://www.ebay.co.uk/itm/334127135022?epid=19023709060&hash=item4dcb87792e:g:MOIAAOSwweNiMHzU&amdata=enc%3AAQAIAAAAwHaBJO8taG6qL42%2BAo%2FQ%2FPPtBN5Y0ANi6KiDmFBzGHDZ9FWSbf1cwJFdrueRobosZIIhgQQaPlq%2BPLLTtOpPLeSnca%2FZ5ait1fT9ROCEHSZhkDSt9zw6CbQ7N707U%2BzMH7Z%2BGrtJCXhxWLvvHXMzu9wCPnpIJ99WlNQKJ5%2Fmve9E0d59Gx%2FG7BZ%2BMNUn10hiqzJEzd7ranOqxfNiQMtd47z8srCSsq9bPVQUgL836rHkyETu3xVyJFhI11A1vaTGUA%3D%3D%7Ctkp%3ABk9SR8idx-vuYg',
 'https://www.ebay.co.uk/itm/266477171796?epid=14023706675&hash=item3e0b46f454:g:uWYAAOSwiv5lPOpN&amdata=enc%3AAQAIAAAA0P4ScB27RVZ7I5978gkvKfypHr8p8Fau%2FpfmcmYxmKvrVk%2FdJQ%2FNQyeeavn1qOcl2W9BRLYBt5gKxIvcyiupG9AxOpj2x%2B1ZyphIB5%2F2Ib3%2FnEAu5nRGBiDCk9w3H1alXFh4tV%2FiFJ45a5LBTqAWvxh%2BU%2FjbHZdTt%2BSFL61iW8evX5xaDcQkwN%2BeHmqnEuF7lifMkLQAQVgENJ8%2FpAVrLOa11O1%2BYbvnkcrHa8QBgrFhOsCiZ7pUu0zgg8dtte4VvSeYgH28gok1hyrmAk%2Bji5U%3D%7Ctkp%3ABFBMurHK6-5i',
 'https://www.ebay.co.uk/itm/283853042331?epid=25046965784&hash=item4216f5b29b:g:gCoAAOSwCsljCeUz&amdata=enc%3AAQAIAAA

In [27]:
# Initialize an empty list to store the results
results = []

for url in urls:
    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the elements
    product_title_element = soup.find('span', {'class': 'ux-textspans ux-textspans--BOLD'})
    #condition_element = soup.find('span', {'data-testid': 'ux-textual-display'})
    price_element = soup.find('div', {'data-testid': 'x-price-primary'})
    #seller_element = soup.find('span', {'class': 'ux-textspans ux-textspans--PSEUDOLINK ux-textspans--BOLD'})
    reviews_element = soup.find('div', {'class': 'x-review-details__allreviews'})
    
    # Get the text from the elements
    product_title = product_title_element.get_text(strip=True)
    #condition = condition_element.find(class_="ux-textspans").text.strip()
    price = price_element.find(class_="ux-textspans").text.strip()
    #seller = seller_element.get_text(strip=True)
    try:
        reviews_link = reviews_element.find('a', {'data-testid': 'ux-action'})['href']
    except:
        continue
    reviews_count = int(reviews_element.find('a', {'data-testid': 'ux-action'}).text.strip().split()[2])
    if reviews_count % 10 == 0:
        pages_count = int(reviews_count/10)
    else:
        pages_count = int(reviews_count/10)+1
    pages_list = list()
    
    for i in range(pages_count):
        page = reviews_link + "&pgn=" + str(i+1)
        response = requests.get(page)
        
        html_content = response.content

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all the review elements
        review_elements = soup.find_all(class_="ebay-review-section")
        

        # Loop through the review elements and extract the text
        for review_element in review_elements:
            try:
                review_text = review_element.find(class_="review-item-content").get_text(strip=True, separator='\n')
                
            except:
                review_text = ""
                pass
            try:
                review_title = review_element.find(class_="review-item-title").get_text(strip=True, separator='\n')
            except:
                review_title = ""
                pass
            try:
                review_star = review_element.find(class_="star-rating")['aria-label']
            except:
                review_star = ""
            try:
                review_author = review_element.find(class_="review-item-author").get_text(strip=True, separator='\n')
            except:
                review_author = ""
                pass
            try:
                review_date = review_element.find(class_="review-item-date").get_text(strip=True, separator='\n')
            except:
                review_date = ""
                pass
            try:
                review_verifiedPurchase = review_element.find_all(class_="rvw-val")[0].get_text(strip=True, separator='\n')
            except:
                review_verifiedPurchase = ""
                pass
            try:
                condition = review_element.find_all(class_="rvw-val")[1].get_text(strip=True, separator='\n')
            except:
                condition = ""
                pass
            try:
                seller = review_element.find_all(class_="rvw-val")[2].get_text(strip=True, separator='\n')
            except:
                seller = ""
                pass
            results.append({'Search': 'Iphone', 'Product Title': product_title, 'Condition': condition, 'Price': price, 'Seller': seller, 
                            'Review': review_text, 'Review - Title': review_title, 'Review - Star': review_star,
                            'Review - Author': review_author, 'Review - Date': review_date,  
                            'Review - Verified Purchase': review_verifiedPurchase})

df = pd.DataFrame(results)
df


Product Title  Condition    Price  \
0   Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
1   Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
2   Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
3   Apple iPhone XR - 64GB - White (Unlocked)        new  £139.99   
4   Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
..                                        ...        ...      ...   
69  Apple iPhone XR - 64GB - White (Unlocked)        new  £139.99   
70  Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
71  Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
72  Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   
73  Apple iPhone XR - 64GB - White (Unlocked)  pre-owned  £139.99   

                Seller                                             Review  \
0               ayyzim  Because it has the size and feel of the I phon...   
1      smellykelly2017  This phone is amazing when I opened it it look...   
2   d63LUdB6TMC@Del...  Good size easy to use and great battery life a...   
3         dsmurpf28028  A fantastic iPhone with a brilliant camera, ba...   
4       tech_outlet_uk  Thank you so much for my new phone looks like ...   
..                 ...                                                ...   
69            mul-1839                                               good   
70         far6mak2012                                          Excellent   
71         nbtechsales                            As described, excellent   
72  mansha-tech-lim...                                               good   
73        hunt4less-uk                                       Superb phone   

   Review - Title Review - Star Review - Author Review - Date  \
0     Great phone       5 stars   herbiebumpkin  13 Jan, 2021   
1     Great phone       5 stars  marekmarek1299  23 Jun, 2020   
2      Nice phone       5 stars        didedo05  21 Aug, 2019   
3     Best iPhone       5 stars        psnape89  14 Oct, 2020   
4   New iPhone XR       5 stars        jaypr-12  11 Jul, 2020   
..            ...           ...             ...           ...   
69           good       4 stars    yogesh151253  21 Feb, 2019   
70      Excellent       5 stars       jasorei-5  12 May, 2020   
71      Excellent       5 stars         rgjandy  07 May, 2019   
72      Thank you       4 stars        alaema-3  01 Jun, 2019   
73     Xr I phone       5 stars    hondacbsteve  24 Aug, 2021   

   Review - Verified Purchase  
0                         Yes  
1                         Yes  
2                         Yes  
3                         Yes  
4                         Yes  
..                        ...  
69                        Yes  
70                        Yes  
71                        Yes  
72                        Yes  
73                        Yes  

[74 rows x 10 columns]